In [1]:
import siesta_utils.grid as siesta 
from siesta_utils.conversions import AtoBohr
import elf 
from ase.io import read, write
import numpy as np 
import xcml
import pickle


In [2]:
atoms = read('./dimer.traj')
dens = elf.real_space.Density(*elf.siesta.get_data('0.RHOXC'))
mask = np.genfromtxt('./O_mask', dtype=bool)
order = np.arange(len(mask))[mask]

basis = {'r_o_o': 1.0,'r_i_o': 0.05, 'r_i_h': 0.0, 'r_o_h' : 1.5,
                      'n_rad_o' : 2,'n_rad_h' : 2, 'n_l_o' : 3,
                      'n_l_h' : 2, 'gamma_o': 0, 'gamma_h': 0}
coeff = elf.real_space.get_elf(atoms, dens, basis)[0]

# elf.geom.make_real_old(coeff)

In [3]:
def rotate_P(P, angles):
    P_rotated = []
    for p in P:
        p_dict = {'0,0,0': 0+0j, '0,1,-1': p[0], '0,1,0': p[1], '0,1,1': p[2]}
        p_rot = elf.geom.rotate_tensor(p_dict, angles)
        p_rot = [p_rot['0,1,{}'.format(m)] for m in [-1,0,1]]
        P_rotated.append(p_rot)
    return np.array(P_rotated)

In [19]:
angles = elf.geom.get_elfcs_angles(coeff, 1, atoms.get_positions())
oriented = elf.geom.rotate_tensor(coeff, angles, inverse = True)
p = elf.geom.tensor_to_P(coeff)
for it in range(100):
    angles = np.random.rand(3)
    coeff2 = elf.geom.rotate_tensor(coeff, angles)
    
    p2 = elf.geom.tensor_to_P(coeff2)
    p_rot = elf.geom.rotate_vector(p, angles)
    assert np.allclose(p2,p_rot)
    pos_rotated = elf.geom.rotate_vector(atoms.get_positions()- atoms.get_positions()[1], angles, False)
#     print(pos_rotated)
    angles = elf.geom.get_elfcs_angles(coeff2, 1, pos_rotated)
    oriented2 = elf.geom.rotate_tensor(coeff2, angles, inverse = True)
    for key in oriented:
        if not np.allclose(oriented[key], oriented2[key],atol = 1e-6):
            raise Exception('error')
# #             print(key)  
# #             print(oriented[key] - oriented2[key])
# #             print(oriented)
# #             print(oriented2)
#             break

Using coordinates for alignment
0.9578367649981014
Using coordinates for alignment
0.9578367649981013
Using coordinates for alignment
0.9578367649981012
Using coordinates for alignment
0.9578367649981014
Using coordinates for alignment
0.9578367649981014
Using coordinates for alignment
0.9578367649981016
Using coordinates for alignment
0.9578367649981014
Using coordinates for alignment
0.9578367649981014
Using coordinates for alignment
0.9578367649981014
Using coordinates for alignment
0.9578367649981014
Using coordinates for alignment
0.957836764998101
Using coordinates for alignment
0.9578367649981014
Using coordinates for alignment
0.9578367649981017
Using coordinates for alignment
0.9578367649981012
Using coordinates for alignment
0.9578367649981012
Using coordinates for alignment
0.9578367649981016
Using coordinates for alignment
0.9578367649981012
Using coordinates for alignment
0.9578367649981013
Using coordinates for alignment
0.957836764998101
Using coordinates for alignment
0

In [11]:
pickle.dump(elf.geom.rotate_tensor(coeff,elf.geom.get_nncs_angles(0,atoms.get_positions()),inverse = True),
            open('elf_nncs.dat', 'wb'))
elf.geom.get_nncs_angles(0,atoms.get_positions())

(-3.0387261472755065, 1.5707963267948966, -1.7982366569471768)

In [12]:

pickle.dump(elf.geom.rotate_tensor(coeff,elf.geom.get_elfcs_angles(coeff,0,atoms.get_positions()),True),
             open('elf_elfcs.dat', 'wb'))
elf.geom.get_elfcs_angles(coeff,0,atoms.get_positions())

(-0.0, 0.0, 0.5217266875033263)

In [9]:
elf.geom.rotate_tensor(coeff,elf.geom.get_nncs_angles(0,atoms.get_positions()), inverse = True)

{'0,0,0': (227.66267531298968+0j),
 '0,1,-1': (1.0512546201400135+10.18352895135375j),
 '0,1,0': (4.3881575177527266+0j),
 '0,1,1': (-1.0512546201400135+10.18352895135375j),
 '0,2,-2': (-2.2594978262072045+0.4715247394586432j),
 '0,2,-1': (-0.060250525113771713-0.583648295166288j),
 '0,2,0': (4.353991418136842-4.440892098500626e-16j),
 '0,2,1': (0.06025052511377704-0.5836482951662879j),
 '0,2,2': (-2.2594978262072045-0.4715247394586437j),
 '1,0,0': (106.98695594997176+0j),
 '1,1,-1': (-0.6972001707192583-6.753794929778951j),
 '1,1,0': (-2.556450327398213+0j),
 '1,1,1': (0.6972001707192583-6.753794929778951j),
 '1,2,-2': (4.618167471485472-0.9637452129900279j),
 '1,2,-1': (0.21223851110586878+2.0559595944044284j),
 '1,2,0': (-6.9455321313730325-6.661338147750939e-16j),
 '1,2,1': (-0.21223851110586356+2.0559595944044293j),
 '1,2,2': (4.618167471485472+0.9637452129900271j)}